# Day 3 - Network basics

## Import libraries

In [ ]:
import pandas as pd
import csv
import glob
import igraph as ig

## Import datasets

In [ ]:
files = sorted(glob.glob('../data/raw/savedrecs_*.txt'))
publications_df = pd.concat(pd.read_csv(f, sep='\t', quoting=csv.QUOTE_NONE, 
                                        usecols=range(68), index_col='UT') for f in files)
publications_df = publications_df.sort_index()

You will see that the data has quite cryptic column headers. Each line contains information about a single publication, and contains various details, such as the title (`TI`), abstract (`AB`), authors (`AU`), journal title (`SO`) and cited references (`CR`). Unfortunately, the documentation of Web of Science is relatively limited, but some explanation can be found <a href="http://images.webofknowledge.com/WOKRS532FR6/help/WOS/hs_advanced_fieldtags.html">here</a>. You can retrieve this information in various ways from the pandas dataframe `publications_df`. For example, you can list the first five titles as follows:

In [ ]:
publications_df.head()

In [ ]:
publications_df.info()

## Data summarisation

The `pandas` package provides various ways to summarise the data and get a useful overview of the data. For example, you can group by a certain column, and count or sum things. For example, we can count the number of articles in each journal that is included in this dataset:

In [ ]:
publications_df.groupby('SO').size().sort_values(ascending=False)[:10]

## Network generation

### Co-authorship

We first build a co-authorship network. We will do this one publication at the time. All combinations of authors that are involved in a publication are co-authors. Let us look at the authors for publication 0.

In [ ]:
publications_df['AU'][0]

Note that the authors are all listed and separated with a semicolon (`;`). In computer terms, it is now a single *string*. We will split this string of all authors into a list of strings where each string then represents a single author.

In [ ]:
publications_df['AU_split'] = publications_df['AU'].fillna('').str.split(';')

In [ ]:
publications_df['AU_split'][0]

Ops... from the second author onwards, the string begins with a whitespace. We need to get rid of that.

In [ ]:
publications_df['AU_split'] = publications_df['AU'].fillna('').str.split(';').apply(lambda lst: [e.strip() for e in lst])
authors = publications_df['AU_split'][0]
authors

In order to create all possible combinations, we can use a convenient package, called `itertools`. The function `combinations` can generate all possible combinations of the elements of a list.

In [ ]:
import itertools as itr
list(itr.combinations(authors, 2))

Of course, we don't want to do this for a single publication only, but rather, for all publications in our dataset. We can do that using the function `apply`.

In [ ]:
coauthors_per_publication = publications_df['AU_split'].apply(lambda authors: list(itr.combinations(authors, 2)))

The variable `coauthors_per_publication` is now a list of a list of co-authors per publication. That is, each element of `coauthors_per_publication` contains a list of all co-authors for that publication. So, `coauthors_per_publication[0]` contains the coauthors we examined previously.

In [ ]:
coauthors_per_publication[0]

In [ ]:
coauthors_per_publication

Let us turn each element of this list into a separate row. This is done by using `explode` in `pandas`. Publications with only one author have no co-authors, which results in an `NA` (Not Available) value. We will drop those using `dropna`.

In [ ]:
coauthors = coauthors_per_publication.explode().dropna()
coauthors

Finally, we can create the actual network as follows

In [ ]:
G_coauthorship = ig.Graph.TupleList(
      edges=coauthors.to_list(),
      vertex_name_attr='author',
      directed=False
      )

Note that this graph will still contain many duplicate edges, because there are multiple edges present. Let us therefore simplify this graph, and simply count the number of multiple edges. We first create a so-called edge attribute `n_joint_papers`. We can create it by using the edge sequence `es` of the graph. We can then simply sum this weight when we simplify the graph.

In [ ]:
G_coauthorship.es['n_joint_papers'] = 1
G_coauthorship = G_coauthorship.simplify(combine_edges='sum')

Let us see how many authors (i.e. nodes) there are in the network. This is called the `vcount` (vertex count) in `igraph`.

In [ ]:
G_coauthorship.vcount()

Similarly, the number of edges is available as the `ecount` of the graph.

In [ ]:
G_coauthorship.ecount()

We can do all sorts of analysis on this network. But first, we will create a bibliographic coupling network.

### Bibliographic coupling

Bibliographic coupling and co-authorship is in a sense very similar. Previously, we computed for each publication a combination of all co-authors. For bibliographic coupling we can compute for each cited reference the combinations of all citing journals. 

We will first create a dataframe of all journal citations (`SO`) of a certain cited reference (`CR`). Similar to the authors, we need to first split the cited references.

In [ ]:
publication_with_cr_df = publications_df[publications_df['CR'].notna()][['SO', 'CR']]
publication_with_cr_df

Now, let us split the CR field into distinct references.

In [ ]:
publication_with_cr_df['CR'] = publication_with_cr_df['CR'].str.split('; ')
publication_with_cr_df

Let's see the references for the first paper

In [ ]:
publication_with_cr_df['CR'][0]

We now simply list all citations from a certain journal (`SO`) to a certain cited reference (`CR`).

In [ ]:
journal_cits_df = publication_with_cr_df[['SO', 'CR']].explode('CR')
journal_cits_df

We then create all bibliographic couplings per cited reference as follows. We first group by the cited reference (`CR`) and then take all combinations of citing journals.

In [ ]:
bibcoupling_per_cr = journal_cits_df.groupby('CR').apply(lambda x: list(itr.combinations(x['SO'], 2)))

We again `explode` all combinations of two sources citing the same reference.

In [ ]:
bibcouplings = bibcoupling_per_cr.explode().dropna()
bibcouplings

We can then create the network.

In [ ]:
G_coupling = ig.Graph.TupleList(
      edges=bibcouplings,
      vertex_name_attr='SO',
      directed=False
      )

<div class="alert alert-info">
    We again need to simplify this network. Create a new edge attribute called <code>coupling</code> set it to <code>1</code> and then sum this attribute when simplifying the network.
</div>

In [ ]:
G_coupling.es['coupling'] = 1
G_coupling = G_coupling.simplify(combine_edges='sum')

This network should be reasonably sized, and you should be able to visualize this network by calling `ig.plot`.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))
ig.plot(G_coupling, vertex_label=G_coupling.vs['SO'], target=ax)

# Network analysis

Now that we have created some scientometric networks, let us look at some basic analyses of these networks.

## Connectivity

Let us start with a very simple question. Is the co-authorship network connected?

In [ ]:
G_coauthorship.is_connected()

Apparently, not all authors in this dataset are connected via co-authored papers.

<div class="alert alert-info">
How many authors do you think will be connected to each other? 500? 5000? Almost everybody?
</div>

In order to take a closer look, we need to detect the *connected components*. This is easily done, but the function is confusingly called `clusters`.

In [ ]:
components = G_coauthorship.connected_components()

In [ ]:
components

Let us only look at the giant component.

In [ ]:
H = components.giant()

Let us check how many nodes are in the giant component. We can call the function `summary`.

In [ ]:
H.summary()

The first line indicates that we have an undirected graph (`U`) with 7871 nodes and 69928 links. The next line shows vertex attributes (indicated by the `v` behind the name of the attribute), and edge attributes (indicated by the `e`).

<div class="alert alert-info">
    <ol>
      <li> What is the percentage of nodes that are in the giant component? 
      <li> Double check whether the giant component is connected.
    </ol>
</div>

Let us take a closer look at how far authors in this data set are apart from one another. Let us simply take a look at node number `0` (remember, the first node has number `0`, not `1`) and node number `355`. 

In [ ]:
paths = G_coauthorship.get_shortest_paths(0, 355)
paths

This returns a list of all shortests paths of the nodes between node number 0 and node number 355. In fact, there is only one path, so let us select that.

In [ ]:
path = paths[0]
path

<div class="alert alert-info">
How many nodes are in the path? What is the path length?
</div>

These numbers probably do not mean that much to you. You can find out more about an individual node by looking at the `VertexSequence` of `igraph`, abbreviated as `vs`. This is a sort of list of all vertices, and is indexed by brackets `[ ]`, similar to lists, instead of parentheses `( )` as we do for functions.

In [ ]:
G_coauthorship.vs[0]

The vertex itself is also a type of list (called a *dictionary*), and you can only return the author name as follows

In [ ]:
G_coauthorship.vs[0]['author']

You can also list multiple vertices at once.

In [ ]:
G_coauthorship.vs[[0, 3, 223, 355]]['author']

You can of course also simply pass the variable `path` that we constructed earlier.

In [ ]:
G_coauthorship.vs[path]['author']

This shows that Osaer collaborated with Geert, who collaborated with Van Mark, who in the end collaborated with Watkins.

You can also get the vertex by searching for the author name. For example, if we want to find `'Van Marck, E'` we can use the following.

In [ ]:
G_coauthorship.vs.find(author_eq = 'Van Marck, E')

Here `author_eq` refers to the condition that the vertex attribute `author` should **eq**ual `'Van Marck, E'`.

<div class="alert alert-info">
    Find the shortest path from <code>'Van Marck, E'</code> to <code>'Migchelsen, S'</code>. Who is in between?
</div>

In [ ]:
G_coauthorship.vs.find(author_eq = 'Migchelsen, S')

In [ ]:
paths = G_coauthorship.get_shortest_paths(223, 3556)
path = paths[0]
G_coauthorship.vs[path]['author']

We can let `igraph` also calculate how far apart all nodes are.

<div class="alert alert-warning">
The following may take some time to run
</div>

In [ ]:
path_lengths = G_coauthorship.path_length_hist()
print(path_lengths)

<div class="alert alert-info">
How far apart are most authors? Do you think most authors are close by? Or do you think they are far apart?
</div>

Let us take a closer look at the path between node 0 and node 355 again. Instead of the nodes on the path, we now want to take a closer look at the edges on the path.

In [ ]:
epath = G_coauthorship.get_shortest_paths(0, 355, output='epath')
epath

There are three edges on this path, but the numbers themselves are not very informative. They refer to the edges, and similar to the `VertexSequence` we encountered earlier, there is also an `EdgeSequence`, abbreviated as `es`. Let us take a closer look to the number of joint papers that the authors had co-authored.

In [ ]:
G_coauthorship.es[epath[0]]['n_joint_papers']

Perhaps there are other paths that connect the two authors with more joint papers? Perhaps we could use the number of joint papers as weights?

In [ ]:
epath = G_coauthorship.get_shortest_paths(0, 355, weights='n_joint_papers', output='epath')
epath

We do get a different path, which it is actually longer. Let us take a look at the number of joint papers.

In [ ]:
G_coauthorship.es[epath[0]]['n_joint_papers']

The total number of joint papers is lower! That is because *shortest path* means: the path with the lowest sum of the weights. This is clearly not what we want. You should always be aware of this whenever using the concept of the *shortest path*.

<div class="alert alert-danger">
<b>Attention!</b> Weighted shortest paths have the <em>lowest</em> total weight.
</div>

## Clustering coefficient

Let us look whether co-authors of an author also tend to be co-authors among themselves.

Let us take a look at the co-authors of of author number 0, which are called the *neighbors* in network terminology.

In [ ]:
G_coauthorship.neighborhood(0)

What we actually want to know is whether many of those neighors are connected. That is, we want to take the subgraph of all authors that have co-authored with author number 0.

In [ ]:
H = G_coauthorship.induced_subgraph(G_coauthorship.neighborhood(0))
H.summary()

This subgraph only has 4 nodes (including node 0, so it has 3 neighbours) and 6 edges. This is sufficiently small to be easily plotted for visual inspection.

In [ ]:
H.vs['color'] = 'grey'
H.vs[0]['color'] = 'red'
fig, ax = plt.subplots()
ig.plot(H, target=ax)

<div class="alert alert-info">
Do many of the co-authors collaborate among themselves as well? Why do you think this happens?
</div>

We can also ask `igraph` to calculate the clustering coefficient (which is called *transitivity* in igraph, which is the same concept using different terms) of node 0.

In [ ]:
G_coauthorship.transitivity_local_undirected(0)

<div class="alert alert-info">
What percentage of the co-authors of node 0 have also written papers with each other?
</div>

You can calculate the average for all nodes using the function `transitivity_avglocal_undirected`.

<div class="alert alert-info">
What percentage of the co-authors have also written papers with each other on average? Do you think this is high or not?
</div>

## Centrality

Often, people want to identify which nodes seem to be most important in some way in the network. This is often thought of as a type of *centrality* of a node.

### Degree

The simplest type of centrality is the *degree* of a node, which is simply the number of its neighbors. Previously, we saw that node 0 had 3 neighbors, we therefore say its degree is 3.

In [ ]:
G_coauthorship.degree(0)

We can also simply calculate the degree for everybody and store it in a new vertex attribute called `degree`.

In [ ]:
G_coauthorship.vs['degree'] = G_coauthorship.degree()

<div class="alert alert-info">
    What is the degree of <code>'Van Marck, E'</code>?
</div>

We can also take a look at the complete degree distribution. To plot it, we load the `matplotlib` package. We import the plotting functionality and name the package `plt`. We also include a statement telling Python to show the plots immediately in this notebook.

Now let us plot a histogram of the degree, using 50 bins.

In [ ]:
plt.hist(G_coauthorship.vs['degree'], 50)
plt.yscale('log')

This clearly shows that the degree distribution is quite skewed. Most authors have only few collaborators, while a few authors have many collaborators. If the degree distribution is so skewed, it is sometimes referred to as a *scale-free* network, although the exact definition has been a topic of intense discussion recently.

The code below sorts the nodes in descending order of the degree.

In [ ]:
highest_degree = sorted(G_coauthorship.vs, key=lambda v: v['degree'], reverse=True)

The `sorted` function takes a list as input, `G_coauthorship.vs` in our case, and sorts it according to a sort key. We indicate the sort key by a small function, called a `lambda` function, that returns the degree. In other words, the `sorted` function will sort the nodes according to the degree. By indicating `reverse=True` we obtain a list that is sorted highest to lowest, instead of the other way around.

You can look at the first five results in the following way.

In [ ]:
highest_degree[:5]

So, apparently, U D'Alessandro has collaborated with 715 other authors! This of course only considers the number of co-authors, it does not take into account the number of papers written with somebody else.
When specifying such *edge weights* like the number of joint papers, the weighted degree is referred to as the *strength* of a node (which is sometimes a bit confusing term). 

Let us look at the strength of node 0.

In [ ]:
G_coauthorship.strength(0, weights='n_joint_papers')

Apparently, author 0 collaborated with 3 different authors, and has a total strength of 3. But what does this 3 mean? We need to carefully think about this. Suppose that author 0 has co-authored a single publication with three other co-authors, then each of the three co-authors would have an edge weight of `n_joint_papers = 1`. So, the strenght would be 3. Hence, the strength denotes the total number of collaborations that an author had, which depends both on the number of publications and the number of collaborators per paper.

Sometimes, we wish to take into account the number of co-authorships when creating a link weight. We can then fractionally count the weight of each collaboration between $n_a$ authors as

$$\frac{1}{n_a - 1}.$$

We need to go back to the `publications_df` in order to construct such a *fractional* edge weight.

In [ ]:
import itertools as itr
[(coauthors[0], coauthors[1], 1/(len(authors) - 1)) for coauthors in itr.combinations(authors, 2)]

We again do this for all publications.

In [ ]:
coauthors_per_publication = publications_df['AU_split'].apply(
    lambda authors: 
        [(coauthor[0], coauthor[1], 1, 1/(len(authors) - 1)) 
             for coauthor in itr.combinations(authors, 2)])

The variable `coauthors_per_publication` is now a list of a list of co-authors per publication, but including both a full weight of `1` and a fractional weight of `1/(len(authors) - 1)`, where `len(authors)` is the number of authors of the publications. We again `explode` this list.

In [ ]:
coauthors = coauthors_per_publication.explode().dropna()
coauthors

We can recreate the network, but now we can pass two edge attributes, `n_joint_papers` and `n_joint_papers_frac`. We of course also have to simplify the network again.

In [ ]:
G_coauthorship = ig.Graph.TupleList(
      edges=coauthors.to_list(),
      vertex_name_attr='author',
      directed=False,
      edge_attrs=('n_joint_papers', 'n_joint_papers_frac')
      )
G_coauthorship = G_coauthorship.simplify(loops=False, combine_edges='sum')

<div class="alert alert-info">
What is the sum of <code>n_joint_papers_frac</code> over all co-authors?
</div>

<div class="alert alert-info">
Shouldn't the strength sum up to a whole number? Why isn't that the case here? (Hint: look at the authors of publication <code>'WOS:000242241600004'</code)
</div>

In [ ]:
publications_df.loc['WOS:000242241600004', 'AU']

An author twice? Really?! You can search online the title of the paper and see what has happened

In [ ]:
publications_df.loc['WOS:000242241600004', 'TI']

### Betweenness centrality

Betweenness centrality is much more elaborate, and gives an indication of the number of times a node is on the shortest path from one node to another node.

As you can imagine, this can take quite some time to calculate for all nodes. We will therefore use the somewhat smaller bibliographic coupling network of journals.

<div class="alert alert-warning">
    <b>Note:</b> On larger networks, it may take a long time to calculate the betweenness centrality.
</div>

In [ ]:
G_coupling.vs['betweenness'] = G_coupling.betweenness()

Now we can look at the journals that have the highest betweenness.

In [ ]:
sorted(G_coupling.vs, key=lambda v: v['betweenness'], reverse=True)[:5]

As we did previously when dealing with shortest paths, we can also use a weight for determining the shortest paths.

In [ ]:
G_coupling.vs['betweenness_weighted'] = G_coupling.betweenness(weights='coupling')

<div class="alert alert-info">
What is journal with the highest weighted betweenness centrality? Does this make sense if you compare it to the unweighted betweenness centrality?
</div>

<div class="alert alert-danger">
    <b>Attention!</b> Weighted shortest paths have the <em>lowest</em> total weight.
</div>

### Pagerank

One way of identifying central nodes relies on the idea of a random walk in a network. We will study this in the journal bibliographic coupling network. When performing such a random walk, we simply go from one journal to the next, following the bibliographic coupling links. The journal that is most frequently visited during such a random walk is then seen as most central. This is actually the idea that underlies Google's famous search engine. Luckily, we can compute that a lot faster than betweenness.

In [ ]:
G_coupling.vs['pagerank'] = G_coupling.pagerank()

<div class="alert alert-info">
Get the top 5 most central journals according to Pagerank. Who is the most central? Are the results very different from the betweenness?
</div>

We can again take into account the weights. In pagerank this means that a journal that is a more closely bibliographically coupled will be more likely to be visited during a random walk. This is actually much more in line with our intuition than the shortest path. Let us see what we get if we do that.

In [ ]:
G_coupling.vs['pagerank_weighted'] = G_coupling.pagerank(weights='coupling')

<div class="alert alert-info">
Get the top 5 central journals again. Are the results different for the weighted version of pagerank?
</div>

In [ ]:
sorted(G_coupling.vs, key=lambda v: v['pagerank_weighted'], reverse=True)[:5]

<div class="alert alert-info">
Pagerank is very similar to the techniques that underly the journal "Eigenfactor" and the "SCImago Journal Rank", which are seen as indicators of the scientific impact of a journal.
</div>

## Co-authorship using bipartite projection

We can also create co-authorship using a more theoretical approach from graph theory. We can first construct a network consisting of publications and authors.

We first again `explode` all authors for each publication, and create a graph out of it.

In [ ]:
author_pubs_df = publications_df['AU_split'].explode()
author_pubs_df

In [ ]:
G_pub_authors = ig.Graph.TupleList(
      edges=author_pubs_df.reset_index().values,
      vertex_name_attr='name',
      directed=False
      )

This network consists of two types: publications and authors. This is called a *bipartite* graph. We can automatically get the types using `is_bipartite`.

In [ ]:
is_bipartite, types = G_pub_authors.is_bipartite(return_types = True)
is_bipartite

The actual types are simply returned as a list of `True` and `False` values, which are arbitrary labels for publications and authors. Let us see what the first label stands for.

In [ ]:
types[0]

In [ ]:
G_pub_authors.vs[0]

From the `name` of node `0` we can see that it refers to a publication, and so `False` indicates publications, while `True` indicates authors.

We now would like to perform a so-called *bipartite projection* onto the authors. This is exactly the type of operation that leads to a co-authorship network. If we were to *project* onto the publication, we would end up with a network of publications where each pair of publications is linked if it is authored by the same author.

In [ ]:
G_author_projection = G_pub_authors.bipartite_projection(types=types, which=True)

By default, it keeps track of the *multiplicity* (i.e. the number of joint papers) in the `weight` edge attribute. Unfortunately, it is not possible to do fractional counting using this approach.

<div class="alert alert-info">
    Check the number of nodes in the bipartite projection. Why is it different from the number of nodes in the earlier constructed <code>G_coauthorship</code>? (Hint: checkout the degree.)
</div>

In [ ]:
G_author_projection.vcount()

Another hint:

In [ ]:
set(G_author_projection.vs['name']) - set(G_coauthorship.vs['author'])